In [1]:
!pip install spotipy
!pip install lyricsgenius


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [29]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials 
client_id = "80230197a428480cbf1c4643273c48ab"
# client_id = "8dc127fb9d144b0199e4a197c578779a"

client_secret = "55594a044c73479580ba20ec56270182"
# client_secret = "e819532817344eecafbd5258469a2af9"

redirect_uri='http://localhost:8888/callback'
# redirect_uri='http://localhost:8889/callback'

# Get the user's authorization
username = '8ev9jb63b564s8oou11rgetbn'

# client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

# OR
client_credentials_manager=SpotifyOAuth(client_id=client_id, client_secret=client_secret, scope="user-top-read", redirect_uri=redirect_uri)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 

In [30]:
import lyricsgenius
genius=lyricsgenius.Genius("jCaG2w1CCDLgiXm8JGamJxOdA1Z8eMEKYJhjecctVLM270hsAJkwDBReNzhbZ4Fg", retries=50)
# artist=genius.search_artist("Taylor Swift", max_songs=3, sort="title")

In [31]:
# get the lists first
# fetch list

import requests
from bs4 import BeautifulSoup

# URL of the website
url = "https://kworb.net/spotify/songs.html"

# Send a GET request to the URL
response = requests.get(url)
content = response.content.decode("utf-8")

# Parse the HTML content of the page
soup = BeautifulSoup(content, "html.parser")

# Find all <td> elements with class "column-song"
song_cells = soup.find_all("td", class_="text")
# song_cells = soup.find_all("td")

artist_list=[]
song_list=[]
total_list=[]

# Extract and print the song titles
for cell in song_cells:
    # Assuming the song title is the text within the <td> element
    content = cell.text.strip()

    if not content.startswith("Various Artists"):
      artist_list.append(content.split(" - ")[0])
      song_list.append(content.split(" - ")[1])
      total_list.append(content)

In [32]:
print(len(artist_list)) # returned 2499 --> good
print(len(song_list)) # returned 2499 --> good
print(len(total_list)) # returned 2499 --> good

2499
2499
2499


In [33]:
# make a dataframe based on the lists

base_df=pd.DataFrame({'artist':artist_list, 'track':song_list, 'total':total_list})
base_df

,artist,track,total
0,The Weeknd,Blinding Lights,The Weeknd - Blinding Lights
1,Ed Sheeran,Shape of You,Ed Sheeran - Shape of You
2,Lewis Capaldi,Someone You Loved,Lewis Capaldi - Someone You Loved
3,Post Malone,Sunflower,Post Malone - Sunflower
4,The Weeknd,Starboy,The Weeknd - Starboy
...,...,...,...
2494,Mora,512,Mora - 512
2495,Myke Towers,Diosa,Myke Towers - Diosa
2496,Vampire Weekend,A-Punk,Vampire Weekend - A-Punk
2497,Fuerza Regida,TQM,Fuerza Regida - TQM


In [34]:
# 3 main objectives: 

# part 1: get track info

import pandas as pd
import lyricsgenius
import json

import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials 

client_id = "80230197a428480cbf1c4643273c48ab"
client_secret = "55594a044c73479580ba20ec56270182"
redirect_uri='http://localhost:8888/callback'

# Get the user's authorization
username = '8ev9jb63b564s8oou11rgetbn'
client_credentials_manager=SpotifyOAuth(client_id=client_id, client_secret=client_secret, scope="user-top-read", redirect_uri=redirect_uri)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 

# for every artist and song, get track info and lyrics (IF POSSIBLE)

# get track information
# Function to search for track information
def get_track_info(artist, track_name):
    results = sp.search(q=f'artist:{artist} track:{track_name}', type='track', limit=1)
    if results['tracks']['items']:
        track_info = results['tracks']['items'][0]
        return {
            'track_name': track_info['name'],
            'artist': track_info['artists'][0]['name'],
            'album': track_info['album']['name'],
            'release_date': track_info['album']['release_date'],
            'duration': track_info['duration_ms'],
            'popularity': track_info['popularity'],
            'preview_url': track_info['preview_url'],
            'explicit': track_info['explicit'],
            'track_ID': track_info['id'],  # Adding Track ID here
        }
    else:
        return None

# Extract track information for each track in the DataFrame
track_info_list = []
for index, row in base_df.iterrows():
    track_info = get_track_info(row['artist'], row['track'])
    if track_info:
        track_info_list.append(track_info)

# Create a new DataFrame with the extracted track information
track_info_df = pd.DataFrame(track_info_list)

# Display the track DataFrame
print(track_info_df)

                                         track_name           artist  \
0                                   Blinding Lights       The Weeknd   
1                                      Shape of You       Ed Sheeran   
2                                 Someone You Loved    Lewis Capaldi   
3     Sunflower - Spider-Man: Into the Spider-Verse      Post Malone   
4                                           Starboy       The Weeknd   
...                                             ...              ...   
2452                                            512             Mora   
2453                                          Diosa      Myke Towers   
2454                                         A-Punk  Vampire Weekend   
2455                                            TQM    Fuerza Regida   
2456                                 NI BIEN NI MAL        Bad Bunny   

                                        album release_date  duration  \
0                                 After Hours   2020-03-20    2

In [35]:
track_info_df

,track_name,artist,album,release_date,duration,popularity,preview_url,explicit,track_ID
0,Blinding Lights,The Weeknd,After Hours,2020-03-20,200040,90,None,False,0VjIjW4GlUZAMYd2vXMi3b
1,Shape of You,Ed Sheeran,÷ (Deluxe),2017-03-03,233712,86,https://p.scdn.co/mp3-preview/7339548839a263fd...,False,7qiZfU4dY1lWllzX7mPBI3
2,Someone You Loved,Lewis Capaldi,Divinely Uninspired To A Hellish Extent,2019-05-17,182160,89,None,False,7qEHsqek33rTcFNT9PFqLf
3,Sunflower - Spider-Man: Into the Spider-Verse,Post Malone,Hollywood's Bleeding,2019-09-06,157560,85,None,False,0RiRZpuVRbi7oqRdSMwhQY
4,Starboy,The Weeknd,Starboy,2016-11-25,230453,91,None,True,7MXVkk9YMctZqd1Srtv4MB
...,...,...,...,...,...,...,...,...,...
2452,512,Mora,Primer Dia de Clases,2021-02-05,193636,75,https://p.scdn.co/mp3-preview/14732f1da793e269...,True,5jzEwSyyymBlf1fa1o39T2
2453,Diosa,Myke Towers,Easy Money Baby,2020-01-24,214013,72,https://p.scdn.co/mp3-preview/0d550502c5c48bf5...,True,3JHpk0DOTOzyh0777JFAky
2454,A-Punk,Vampire Weekend,Vampire Weekend,2008-01-29,137760,70,https://p.scdn.co/mp3-preview/3afe492f2ee66f7a...,False,3AydAydLzyyZutA0375XIz
2455,TQM,Fuerza Regida,TQM,2023-05-19,158965,78,https://p.scdn.co/mp3-preview/63f6b66cfcc3d7bd...,True,368eeEO3Y2uZUQ6S5oIjcu


In [ ]:
# # lyrics 

# import lyricsgenius
# import pandas as pd

# # Initialize the Genius API with your access token
# genius=lyricsgenius.Genius("jCaG2w1CCDLgiXm8JGamJxOdA1Z8eMEKYJhjecctVLM270hsAJkwDBReNzhbZ4Fg", retries=3)

# # Function to search for lyrics
# def get_lyrics(artist, track_name):
#     song = genius.search_song(track_name, artist)
#     if song:
#         return song.lyrics
#     else:
#         return None

# # Extract lyrics for each track in the DataFrame
# lyrics_list = []
# for index, row in track_info_df.iterrows():
#     print(row['track_name'])
#     lyrics = get_lyrics(row['artist'], row['track_name'])
#     if lyrics:
#         lyrics_list.append(lyrics)

# # Display the lyrics
# for lyrics in lyrics_list:
#     print(lyrics)

# # Create a new DataFrame with the extracted track information
# lyrics_df = pd.DataFrame(lyrics_list)

# # Display the track DataFrame
# print(lyrics_df)


In [40]:
# lyrics 

import lyricsgenius
import pandas as pd

# Initialize the Genius API with your access token
genius=lyricsgenius.Genius("jCaG2w1CCDLgiXm8JGamJxOdA1Z8eMEKYJhjecctVLM270hsAJkwDBReNzhbZ4Fg", retries=3)

# Function to search for lyrics
def get_lyrics(artist, track_name):
    song = genius.search_song(track_name, artist)
    if song:
        return song.lyrics
    else:
        return None

# Add a new column 'lyrics' to track_info_df
track_info_df['lyrics'] = None

# Extract lyrics for each track in the DataFrame
for index, row in track_info_df.iterrows():
    print(row['track_name'])
    lyrics = get_lyrics(row['artist'], row['track_name'])
    track_info_df.at[index, 'lyrics'] = lyrics  # Assign lyrics even if None

# Display the updated track_info_df with lyrics
print(track_info_df)


Blinding Lights
Searching for "Blinding Lights" by The Weeknd...
Done.
Shape of You
Searching for "Shape of You" by Ed Sheeran...
Done.
Someone You Loved
Searching for "Someone You Loved" by Lewis Capaldi...
Done.
Sunflower - Spider-Man: Into the Spider-Verse
Searching for "Sunflower - Spider-Man: Into the Spider-Verse" by Post Malone...
Done.
Starboy
Searching for "Starboy" by The Weeknd...
Done.
As It Was
Searching for "As It Was" by Harry Styles...
Done.
One Dance
Searching for "One Dance" by Drake...
Done.
Dance Monkey
Searching for "Dance Monkey" by Tones And I...
Done.
STAY (with Justin Bieber)
Searching for "STAY (with Justin Bieber)" by The Kid LAROI...
Done.
Believer
Searching for "Believer" by Imagine Dragons...
Done.
rockstar (feat. 21 Savage)
Searching for "rockstar (feat. 21 Savage)" by Post Malone...
Done.
Heat Waves
Searching for "Heat Waves" by Glass Animals...
Done.
Perfect
Searching for "Perfect" by Ed Sheeran...
Done.
Closer
Searching for "Closer" by The Chainsmokers

In [ ]:
# a

print(track_info_df)

In [54]:
# read the last 56

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time

# Function to fetch audio features for a list of track IDs with exponential backoff
def get_audio_features_batch(track_ids):
    max_retries = 3  # Maximum number of retries
    delay = 1  # Initial delay in seconds
    for i in range(max_retries):
        try:
            features_batch = sp.audio_features(tracks=track_ids)
            return features_batch
        except spotipy.SpotifyException as e:
            # Check if the error is due to rate limiting (status code 429)
            if e.http_status == 429:
                # Calculate backoff time
                backoff_time = 2 ** i * delay
                retry_after = int(e.headers.get('Retry-After', 1))
                print(f"Retry-After header: {retry_after} seconds")
                print(f"Waiting for {backoff_time} seconds before retrying...")
                time.sleep(backoff_time)
            else:
                # Re-raise the exception if it's not a rate limit error
                raise e
    return None

# Extract track IDs from the DataFrame
track_ids = track_info_df['track_ID'].tolist()

# print(track_ids[2400:2457])

# Batch size for batch requests
# batch_size = 100
# num_batches = len(track_ids) // batch_size

# Iterate over batches of track IDs
batch_track_ids = track_ids[2400:2457]
batch_audio_features = get_audio_features_batch(batch_track_ids)
# Update original DataFrame with audio features
for index, audio_feature in zip(range(2400, 2457), batch_audio_features):
    for key, value in audio_feature.items():
        track_info_df.at[index, key] = value

# Display the updated DataFrame with audio features
print(track_info_df)


['5BSclXJTa9B0iURhUjZo50', '1fQK5r0Sj5QUlczS3inV9f', '5kCoykriGdZhWMlm9vrtQI', '2epbL7s3RFV81K5UhTgZje', '5z6xHjCZr7a7AIcy8sPBKy', '0KQx6HOpJueiSkztcS0r7D', '2xql0pid3EUwW38AsywxhV', '5Y35SjAfXjjG0sFQ3KOxmm', '51YZAJhOwIC5Gg3jMbAmhZ', '1JO1xLtVc8mWhIoE3YaCL0', '0HlMshB5JmZjPNbOuOgFHN', '2qQpFbqqkLOGySgNK8wBXt', '4NpDZPwSXmL0cCTaJuVrCw', '03UrZgTINDqvnUMbbIMhql', '5ZBeML7Lf3FMEVviTyvi8l', '2Gl0FzuLxflY6nPifJp5Dr', '2kKkl59fY6Cic1CmhvSEZK', '08H5Y8ySUpGUhjb67cK8S5', '2oaK4JLVnmRGIO9ytBE1bt', '452KBpASS5lZLeJWX9Ixub', '7oVEtyuv9NBmnytsCIsY5I', '4EKZsrsCKyqr64FBHLc0DU', '7ywzEdEuVG1lsjsRCGJfLy', '14rZjW3RioG7WesZhYESso', '2vgUijXOTRMnWXDtvgMp2b', '5w40ZYhbBMAlHYNDaVJIUu', '52ojopYMUzeNcudsoz7O9D', '39KG4kom3enSx4GTThuDGt', '2t2xavvQ8B1CvgooGJEme5', '3Te8uLyit6X3ncNW8Fp3K2', '0UbesRsX2TtiCeamOIVEkp', '6QeYSvYqYUsfBzsApbjDHO', '3CYH422oy1cZNoo0GTG1TK', '4TGwERXRlyQtBdggYTHo6j', '4uhvMW7ly7tJil31YYscAN', '6CFPFnS9EcLs2I0nWqtWci', '54X78diSLoUDI3joC2bjMz', '776AftMmFFAWUIEAb3lHhw', '1WsEgieHsW

In [49]:
track_info_df.tail(60)

,track_name,artist,album,release_date,duration,popularity,preview_url,explicit,track_ID,lyrics,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
2397,Post to Be (feat. Chris Brown & Jhene Aiko),Omarion,Sex Playlist,2014-12-02,226580,69,https://p.scdn.co/mp3-preview/6e46983bc460a046...,True,0fgZUSa7D7aVvv3GfO0A1n,2 ContributorsChris Brown Videography Lyrics20...,...,0.2080,0.701,97.448,audio_features,0fgZUSa7D7aVvv3GfO0A1n,spotify:track:0fgZUSa7D7aVvv3GfO0A1n,https://api.spotify.com/v1/tracks/0fgZUSa7D7aV...,https://api.spotify.com/v1/audio-analysis/0fgZ...,226581.0,4.0
2398,From Time,Drake,Nothing Was The Same (Deluxe),2013-01-01,322160,68,None,True,2WC4sK0ryyysQhtDok9Ytr,225 ContributorsTranslationsTürkçeEspañolFrom ...,...,0.1260,0.440,94.485,audio_features,2WC4sK0ryyysQhtDok9Ytr,spotify:track:2WC4sK0ryyysQhtDok9Ytr,https://api.spotify.com/v1/tracks/2WC4sK0ryyys...,https://api.spotify.com/v1/audio-analysis/2WC4...,322160.0,4.0
2399,We Own It (Fast & Furious),2 Chainz,B.O.A.T.S. II #METIME (Deluxe),2013-01-01,227906,69,None,False,2OQJKTtrH482waGFmOfJni,143 ContributorsWe Own It (Fast & Furious) Lyr...,...,0.0524,0.559,171.999,audio_features,2OQJKTtrH482waGFmOfJni,spotify:track:2OQJKTtrH482waGFmOfJni,https://api.spotify.com/v1/tracks/2OQJKTtrH482...,https://api.spotify.com/v1/audio-analysis/2OQJ...,227907.0,4.0
2400,La Tortura (feat. Alejandro Sanz),Shakira,"Fijación Oral, Vol. 1",2005-06-03,212893,77,https://p.scdn.co/mp3-preview/5a18716291099508...,False,5BSclXJTa9B0iURhUjZo50,4 ContributorsTranslationsEuskaraShakira - La ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2401,What's Love Got to Do with It - Symphonic Version,Tina Turner,What's Love Got to Do with It (Symphonic Version),2018-11-09,256720,20,https://p.scdn.co/mp3-preview/26476dfa535ebff5...,False,1fQK5r0Sj5QUlczS3inV9f,1 ContributorWhat’s Love Got to Do with It (Sy...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2402,Toda - Remix,Alex Rose,ENR,2022-06-09,368453,69,https://p.scdn.co/mp3-preview/51d496e8d5305917...,False,5kCoykriGdZhWMlm9vrtQI,"32 ContributorsToda (Remix) Lyrics[Letra de ""T...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2403,Lemon Tree,Fools Garden,Dish of the Day,1995-01-01,191026,76,https://p.scdn.co/mp3-preview/c0727bb5b3d094a8...,False,2epbL7s3RFV81K5UhTgZje,42 ContributorsTranslationsDeutschLemon Tree L...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2404,Smooth Criminal,Alien Ant Farm,Anthology,2001-01-01,209266,76,None,False,5z6xHjCZr7a7AIcy8sPBKy,30 ContributorsSmooth Criminal Lyrics[Intro]\n...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2405,Fallin',Alicia Keys,Songs In A Minor,2001-06-05,210200,76,https://p.scdn.co/mp3-preview/79526addc56f4ca7...,False,0KQx6HOpJueiSkztcS0r7D,49 ContributorsFallin’ Lyrics[Verse 1]\nI keep...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2406,Reflections,The Neighbourhood,Hard To Imagine The Neighbourhood Ever Changing,2018-11-02,244013,83,https://p.scdn.co/mp3-preview/a0fbb407068bf547...,False,2xql0pid3EUwW38AsywxhV,33 ContributorsReflections Lyrics[Verse 1]\nWh...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
# export to csv
track_info_df.to_csv('sample.csv', index=False)


In [24]:
# audio features (HAS TO BE SEPARATED)

# Add audio features (hopefully API works now...)

# Function to fetch audio features for a list of track IDs with exponential backoff
def get_audio_features_batch(track_ids):
    max_retries = 3  # Maximum number of retries
    delay = 1  # Initial delay in seconds
    for i in range(max_retries):
        try:
            features_batch = sp.audio_features(tracks=track_ids)
            return features_batch
        except spotipy.SpotifyException as e:
            # Check if the error is due to rate limiting (status code 429)
            if e.http_status == 429:
                # Calculate backoff time
                backoff_time = 2 ** i * delay
                retry_after = int(e.headers.get('Retry-After', 1))
                print(f"Retry-After header: {retry_after} seconds")
                print(f"Waiting for {backoff_time} seconds before retrying...")
                time.sleep(backoff_time)
            else:
                # Re-raise the exception if it's not a rate limit error
                raise e
    return None

# Extract track IDs from the DataFrame
track_ids = lyrics_df['track_ID'].tolist()

# Batch size for batch requests
batch_size = 100
num_batches = len(track_ids) // batch_size
# num_batches = 10

# List to store audio features
audio_features = []

# Iterate over batches of track IDs
for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(track_ids))
    batch_track_ids = track_ids[start_idx:end_idx]
    batch_audio_features = get_audio_features_batch(batch_track_ids)
    audio_features.extend(batch_audio_features)

# Convert the list of audio features to a DataFrame
audio_features_df = pd.DataFrame(audio_features)

# Merge audio features DataFrame with the original DataFrame based on index
full_df = pd.concat([lyrics_df, audio_features_df], axis=1)

# Display the updated DataFrame
print(full_df)


                                         track_name           artist  \
0                                   Blinding Lights       The Weeknd   
1                                      Shape of You       Ed Sheeran   
2                                 Someone You Loved    Lewis Capaldi   
3     Sunflower - Spider-Man: Into the Spider-Verse      Post Malone   
4                                           Starboy       The Weeknd   
...                                             ...              ...   
2452                                         A-Punk  Vampire Weekend   
2453                                            TQM    Fuerza Regida   
2454                                 NI BIEN NI MAL        Bad Bunny   
2455         Dark Knight Dummo (feat. Travis Scott)     Trippie Redd   
2456                                      The Bones     Maren Morris   

                                        album release_date  duration  \
0                                 After Hours   2020-03-20    2

In [25]:
# export to csv
full_df.to_csv('full.csv', index=False)

In [25]:
import pandas as pd
import lyricsgenius
import json

import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials 
client_id = "80230197a428480cbf1c4643273c48ab"
# client_id = "8dc127fb9d144b0199e4a197c578779a"

client_secret = "55594a044c73479580ba20ec56270182"
# client_secret = "6c5a1a735be9413da3d546dd94f5a16c"

redirect_uri='http://localhost:8888/callback'
# redirect_uri='http://localhost:8889/callback'

# Get the user's authorization
username = '8ev9jb63b564s8oou11rgetbn'

# client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

# OR
client_credentials_manager=SpotifyOAuth(client_id=client_id, client_secret=client_secret, scope="user-top-read", redirect_uri=redirect_uri)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 

# Assuming you have initialized the Spotify client (sp) elsewhere in your code
# Assuming you have obtained and set up the Genius API credentials elsewhere in your code

# Genius API credentials
# genius = lyricsgenius.Genius("YOUR_GENIUS_API_KEY_HERE")

# Maximum number of retries
max_retries = 3

# Sleep time between retries (in seconds)
sleep_time = 10

# Function to search for song lyrics with retries
def search_song_with_retries(genius, song_name, artist_name):
    retries = 0
    while retries < max_retries:
        try:
            # Attempt to search for the song
            song = genius.search_song(song_name, artist_name)
            if song:
                return song.lyrics
        except Exception as e:
            print("Error occurred:", e)
        # Song not found or error occurred, calculate backoff time
        backoff_time = 2 ** retries * sleep_time
        print(f"Waiting for {backoff_time} seconds before retrying...")
        time.sleep(backoff_time)
        retries += 1
    return None

# getting top 10 songs for the top 100 artist 
artist_list = [
    "Taylor Swift", "Drake", "Bad Bunny", "The Weeknd", "Ed Sheeran",
    "Ariana Grande", "Justin Bieber", "Eminem", "Post Malone", "BTS",
    "Kanye West", "J Balvin", "Billie Eilish", "Coldplay", "Travis Scott",
    "Dua Lipa", "Juice WRLD", "Rihanna", "Imagine Dragons", "XXXTENTACION",
    "Ozuna", "David Guetta", "Maroon 5", "Bruno Mars", "Khalid",
    "Kendrick Lamar", "Shawn Mendes", "Calvin Harris", "Sam Smith",
    "Lana Del Rey", "Beyoncé", "Daddy Yankee", "KAROL G", "Queen",
    "Harry Styles", "Future", "Lady Gaga", "One Direction", "Nicki Minaj",
    "J. Cole", "Adele", "Rauw Alejandro", "Anuel AA", "Chris Brown",
    "Shakira", "The Beatles", "The Chainsmokers", "Selena Gomez",
    "Linkin Park", "Arctic Monkeys", "Doja Cat", "Lil Uzi Vert", "Sia",
    "SZA", "Marshmello", "Maluma", "Halsey", "Katy Perry", "Metro Boomin",
    "21 Savage", "Miley Cyrus", "Olivia Rodrigo", "Twenty One Pilots",
    "Avicii", "Arijit Singh", "Red Hot Chili Peppers", "Lil Baby", "Kygo",
    "Feid", "Farruko", "OneRepublic", "Camila Cabello", "Myke Towers",
    "Morgan Wallen", "Michael Jackson", "Peso Pluma", "Jason Derulo",
    "Tyler, The Creator", "$uicideboy$", "Pitbull", "Mac Miller",
    "Pritam", "Elton John", "Demi Lovato", "Metallica", "Pop Smoke",
    "BLACKPINK", "Frank Ocean", "Tiësto", "Ellie Goulding", "DJ Snake",
    "Alan Walker", "Lil Wayne", "The Neighbourhood", "James Arthur",
    "DaBaby", "YoungBoy Never Broke Again", "AC/DC", "Charlie Puth",
    "Lil Nas X"
]

# Create an empty list to store track information
track_data = []

for artist_name in artist_list:
    # Search for the artist
    results = sp.search(q=artist_name, type='artist')

    # Check if any artist was found
    if len(results['artists']['items']) > 0:
        # Get the first artist found
        artist = results['artists']['items'][0]

        # Get the top tracks for the artist
        top_tracks = sp.artist_top_tracks(artist['id'])

        # Populate the DataFrame with track information
        for track in top_tracks['tracks'][:10]:
            # Get audio features for the track
            # audio_features = sp.audio_features(track['id'])[0]  # Assuming the first track in the list

            # Search for the lyrics
            # lyrics = search_song_with_retries(genius, track['name'], artist_name)

            # Append track information to list
            track_data.append({
                'Artist': artist_name,
                'Track': track['name'],
                'Track ID': track['id'],  # Adding Track ID here
                'Popularity': track['popularity'],
                'Duration': track['duration_ms'],
                'Release Date': track['album']['release_date'],
                'Album Name': track['album']['name'],
                'Explicit': track['explicit'],


                # 'Audio': track['preview_url'],
                # 'Cover Art': track['album']['images'][0]['url'],
                # 'Danceability': audio_features['danceability'],
                # 'Energy': audio_features['energy'],
                # 'Key': audio_features['key'],
                # 'Loudness': audio_features['loudness'],
                # 'Mode': audio_features['mode'],
                # 'Speechiness': audio_features['speechiness'],
                # 'Acousticness': audio_features['acousticness'],
                # 'Instrumentalness': audio_features['instrumentalness'],
                # 'Liveness': audio_features['liveness'],
                # 'Valence': audio_features['valence'],
                # 'Tempo': audio_features['tempo'],
                # 'Duration_ms': audio_features['duration_ms'],
                # 'Lyrics': lyrics
            })
    else:
        print("No artist found with the name:", artist_name)

# Create DataFrame from track data
track_df = pd.DataFrame(track_data)

# Print the DataFrame
print(track_df)


           Artist                                              Track  \
0    Taylor Swift                                       Cruel Summer   
1    Taylor Swift                                              Lover   
2    Taylor Swift                                          Anti-Hero   
3    Taylor Swift                                           cardigan   
4    Taylor Swift  Is It Over Now? (Taylor's Version) (From The V...   
..            ...                                                ...   
995     Lil Nas X                                           J CHRIST   
996     Lil Nas X                                      Old Town Road   
997     Lil Nas X                                             Panini   
998     Lil Nas X                                            HOLIDAY   
999     Lil Nas X                                              Rodeo   

                   Track ID  Popularity  Duration Release Date  \
0    1BxfuPKGuaTgP7aM0Bbdwr          97    178426   2019-08-23   
1  

In [29]:
track_df.head()


,Artist,Track,Track ID,Popularity,Duration,Release Date,Album Name,Explicit
0,Taylor Swift,Cruel Summer,1BxfuPKGuaTgP7aM0Bbdwr,97,178426,2019-08-23,Lover,False
1,Taylor Swift,Lover,1dGr1c8CrMLDpV6mPbImSI,90,221306,2019-08-23,Lover,False
2,Taylor Swift,Anti-Hero,0V3wPSX9ygBnCm8psDIegu,89,200690,2022-10-21,Midnights,False
3,Taylor Swift,cardigan,4R2kfaDFhslZEMJqAFNpdd,88,239560,2020-07-24,folklore,False
4,Taylor Swift,Is It Over Now? (Taylor's Version) (From The V...,1Iq8oo9XkmmvCQiGOfORiz,88,229477,2023-10-26,1989 (Taylor's Version),False


In [32]:
import requests
import pandas as pd
import time
import lyricsgenius

genius=lyricsgenius.Genius("jCaG2w1CCDLgiXm8JGamJxOdA1Z8eMEKYJhjecctVLM270hsAJkwDBReNzhbZ4Fg", retries=50)

# Maximum number of retries
max_retries = 2

# Sleep time between retries (in seconds)
sleep_time = 10

# Function to search for song lyrics with retries
def search_song_with_retries(genius, song_name, artist_name):
    retries = 0
    while retries < max_retries:
        try:
            # Attempt to search for the song
            song = genius.search_song(song_name, artist_name)
            if song:
                return song.lyrics
        except Exception as e:
            print("Error occurred:", e)
        # Song not found or error occurred, calculate backoff time
        backoff_time = 2 ** retries * sleep_time
        print(f"Waiting for {backoff_time} seconds before retrying...")
        time.sleep(backoff_time)
        retries += 1
    return None

# Assuming df is your DataFrame with track and artist information

# Replace 'access_token' with your actual Genius API access token

# Create a new column to store lyrics
track_df['Lyrics'] = ''

# Iterate over rows in DataFrame
for index, row in track_df.iterrows():
    track_name = row['Track']
    artist_name = row['Artist']
    lyrics = search_song_with_retries(genius, track_name, artist_name)
    # Add lyrics to DataFrame
    track_df.at[index, 'Lyrics'] = lyrics

# Display the updated DataFrame
print(track_df)

Searching for "Cruel Summer" by Taylor Swift...
Done.
Searching for "Lover" by Taylor Swift...
Done.
Searching for "Anti-Hero" by Taylor Swift...
Done.
Searching for "cardigan" by Taylor Swift...
Done.
Searching for "Is It Over Now? (Taylor's Version) (From The Vault)" by Taylor Swift...
Done.
Searching for "Don’t Blame Me" by Taylor Swift...
Done.
Searching for "august" by Taylor Swift...
Done.
Searching for "Karma" by Taylor Swift...
Done.
Searching for "Love Story (Taylor’s Version)" by Taylor Swift...
Done.
Searching for "Blank Space" by Taylor Swift...
Done.
Searching for "Rich Baby Daddy (feat. Sexyy Red & SZA)" by Drake...
Done.
Searching for "One Dance" by Drake...
Done.
Searching for "IDGAF (feat. Yeat)" by Drake...
Done.
Searching for "First Person Shooter (feat. J. Cole)" by Drake...
Done.
Searching for "Jimmy Cooks (feat. 21 Savage)" by Drake...
Done.
Searching for "act ii: date @ 8 (feat. Drake) - remix" by Drake...
Done.
Searching for "God's Plan" by Drake...
Done.
Search

In [37]:
track_df.head()

,Artist,Track,Track ID,Popularity,Duration,Release Date,Album Name,Explicit,Lyrics
0,Taylor Swift,Cruel Summer,1BxfuPKGuaTgP7aM0Bbdwr,97,178426,2019-08-23,Lover,False,203 ContributorsTranslationsTürkçeEspañolСрпск...
1,Taylor Swift,Lover,1dGr1c8CrMLDpV6mPbImSI,90,221306,2019-08-23,Lover,False,176 ContributorsTranslationsРусскийالعربيةУкра...
2,Taylor Swift,Anti-Hero,0V3wPSX9ygBnCm8psDIegu,89,200690,2022-10-21,Midnights,False,200 ContributorsTranslationsPortuguêsEspañolTü...
3,Taylor Swift,cardigan,4R2kfaDFhslZEMJqAFNpdd,88,239560,2020-07-24,folklore,False,287 ContributorsTranslationsNederlandsTürkçeEs...
4,Taylor Swift,Is It Over Now? (Taylor's Version) (From The V...,1Iq8oo9XkmmvCQiGOfORiz,88,229477,2023-10-26,1989 (Taylor's Version),False,160 ContributorsTranslationsPortuguêsEspañolFr...
...,...,...,...,...,...,...,...,...,...
995,Lil Nas X,J CHRIST,17CWWoszMuoGecCJzmOubE,74,153766,2024-01-12,J CHRIST,True,65 ContributorsTranslationsDeutschБългарскиEsp...
996,Lil Nas X,Old Town Road,0F7FA14euOIX8KcbEturGH,74,113000,2019-06-21,7 EP,False,256 ContributorsTranslationsEspañolРусскийفارس...
997,Lil Nas X,Panini,1000nHvUdawXuUHgBod4Wv,68,114893,2019-09-13,Panini,False,157 ContributorsTranslationsEspañolРусскийPort...
998,Lil Nas X,HOLIDAY,6zFMeegAMYQo0mt8rXtrli,69,154997,2020-11-13,HOLIDAY,True,183 ContributorsTranslationsTürkçeEspañolDeuts...


In [52]:


print(sp.audio_features(['1dGr1c8CrMLDpV6mPbImSI']))


[{'danceability': 0.359, 'energy': 0.543, 'key': 7, 'loudness': -7.582, 'mode': 1, 'speechiness': 0.0919, 'acousticness': 0.492, 'instrumentalness': 1.58e-05, 'liveness': 0.118, 'valence': 0.453, 'tempo': 68.534, 'type': 'audio_features', 'id': '1dGr1c8CrMLDpV6mPbImSI', 'uri': 'spotify:track:1dGr1c8CrMLDpV6mPbImSI', 'track_href': 'https://api.spotify.com/v1/tracks/1dGr1c8CrMLDpV6mPbImSI', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1dGr1c8CrMLDpV6mPbImSI', 'duration_ms': 221307, 'time_signature': 4}]


In [53]:
# Add audio features (hopefully API works now...)


# Function to fetch audio features for a list of track IDs with exponential backoff
def get_audio_features_batch(track_ids):
    max_retries = 3  # Maximum number of retries
    delay = 1  # Initial delay in seconds
    for i in range(max_retries):
        try:
            features_batch = sp.audio_features(tracks=track_ids)
            return features_batch
        except spotipy.SpotifyException as e:
            # Check if the error is due to rate limiting (status code 429)
            if e.http_status == 429:
                # Calculate backoff time
                backoff_time = 2 ** i * delay
                retry_after = int(e.headers.get('Retry-After', 1))
                print(f"Retry-After header: {retry_after} seconds")
                print(f"Waiting for {backoff_time} seconds before retrying...")
                time.sleep(backoff_time)
            else:
                # Re-raise the exception if it's not a rate limit error
                raise e
    return None

# Load your DataFrame
# df = pd.read_csv('your_dataframe.csv')

# Rename the DataFrame to track_df
# track_df = df.copy()

# Extract track IDs from the DataFrame
track_ids = track_df['Track ID'].tolist()

# Batch size for batch requests
batch_size = 100
num_batches = len(track_ids) // batch_size
# num_batches = 10

# List to store audio features
audio_features = []

# Iterate over batches of track IDs
for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(track_ids))
    batch_track_ids = track_ids[start_idx:end_idx]
    batch_audio_features = get_audio_features_batch(batch_track_ids)
    audio_features.extend(batch_audio_features)

# Convert the list of audio features to a DataFrame
audio_features_df = pd.DataFrame(audio_features)

# Merge audio features DataFrame with the original DataFrame based on index
track_df = pd.concat([track_df, audio_features_df], axis=1)

# Display the updated DataFrame
print(track_df)


           Artist                                              Track  \
0    Taylor Swift                                       Cruel Summer   
1    Taylor Swift                                              Lover   
2    Taylor Swift                                          Anti-Hero   
3    Taylor Swift                                           cardigan   
4    Taylor Swift  Is It Over Now? (Taylor's Version) (From The V...   
..            ...                                                ...   
995     Lil Nas X                                           J CHRIST   
996     Lil Nas X                                      Old Town Road   
997     Lil Nas X                                             Panini   
998     Lil Nas X                                            HOLIDAY   
999     Lil Nas X                                              Rodeo   

                   Track ID  Popularity  Duration Release Date  \
0    1BxfuPKGuaTgP7aM0Bbdwr          97    178426   2019-08-23   
1  

In [54]:
# export to csv
track_df.to_csv('test.csv', index=False)

In [23]:
# import sys

# import time

# # Maximum number of retries
# max_retries = 5

# # Sleep time between retries (in seconds)
# sleep_time = 10

# # Function to search for song lyrics with retries
# def search_song_with_retries(genius, song_name, artist_name):
#     retries = 0
#     while retries < max_retries:
#         try:
#             # Attempt to search for the song
#             song = genius.search_song(song_name, artist_name)
#             if song:
#                 return song.lyrics
#         except Exception as e:
#             print("Error occurred:", e)
#         # Song not found or error occurred, calculate backoff time
#         backoff_time = 2 ** retries * sleep_time
#         print(f"Waiting for {backoff_time} seconds before retrying...")
#         time.sleep(backoff_time)
#         retries += 1
#     return None

# def get_audio_features_with_retries(sp, track_id):
#     retries = 0
#     while retries < max_retries:
#         try:
#             # Attempt to retrieve audio features for the track
#             audio_features = sp.audio_features(track_id)[0]  # Assuming the first track in the list
#             return audio_features
#         except spotipy.SpotifyException as e:
#             # Check if the error is due to rate limiting (status code 429)
#             if e.http_status == 429:
#                 # Calculate backoff time
#                 backoff_time = 2 ** retries * sleep_time
#                 retry_after=int(e.response.headers.get('Retry-After', 1))
#                 print(f"The retry time is: {retry_after}")
#                 print(f"Waiting for {backoff_time} seconds before retrying...")
#                 time.sleep(backoff_time)
#                 retries += 1
#             else:
#                 # Re-raise the exception if it's not a rate limit error
#                 raise e
        
#     # Return None if maximum retries exceeded
#     return None


# # getting top 10 songs for the top 100 artist 
# # artist_list=['Yoasobi']

# artist_list = [
#     "Taylor Swift", "Drake", "Bad Bunny", "The Weeknd", "Ed Sheeran",
#     "Ariana Grande", "Justin Bieber", "Eminem", "Post Malone", "BTS",
#     "Kanye West", "J Balvin", "Billie Eilish", "Coldplay", "Travis Scott",
#     "Dua Lipa", "Juice WRLD", "Rihanna", "Imagine Dragons", "XXXTENTACION",
#     "Ozuna", "David Guetta", "Maroon 5", "Bruno Mars", "Khalid",
#     "Kendrick Lamar", "Shawn Mendes", "Calvin Harris", "Sam Smith",
#     "Lana Del Rey", "Beyoncé", "Daddy Yankee", "KAROL G", "Queen",
#     "Harry Styles", "Future", "Lady Gaga", "One Direction", "Nicki Minaj",
#     "J. Cole", "Adele", "Rauw Alejandro", "Anuel AA", "Chris Brown",
#     "Shakira", "The Beatles", "The Chainsmokers", "Selena Gomez",
#     "Linkin Park", "Arctic Monkeys"
# ]

# track_data = []

# for artist_name in artist_list:
#     # Search for the artist
#     results = sp.search(q=artist_name, type='artist')

#     # Check if any artist was found
#     if len(results['artists']['items']) > 0:
#         # Get the first artist found
#         artist = results['artists']['items'][0]

#         # Get the top tracks for the artist
#         top_tracks = sp.artist_top_tracks(artist['id'])

#         # Populate the DataFrame with track information
#         for track in top_tracks['tracks'][:10]:
#             # Get audio features for the track with retries
#             audio_features = get_audio_features_with_retries(sp, track['id'])
            
#             # If audio features are retrieved successfully, continue processing
#             if audio_features:
#                 # Search for the lyrics with retries
#                 lyrics = search_song_with_retries(genius, track['name'], artist_name)

#                 # Append track information to list
#                 track_data.append({
#                     'Artist': artist_name,
#                     'Track': track['name'],
#                     'Danceability': audio_features['danceability'],
#                     'Energy': audio_features['energy'],
#                     'Key': audio_features['key'],
#                     'Loudness': audio_features['loudness'],
#                     'Mode': audio_features['mode'],
#                     'Speechiness': audio_features['speechiness'],
#                     'Acousticness': audio_features['acousticness'],
#                     'Instrumentalness': audio_features['instrumentalness'],
#                     'Liveness': audio_features['liveness'],
#                     'Valence': audio_features['valence'],
#                     'Tempo': audio_features['tempo'],
#                     'Duration_ms': audio_features['duration_ms'],
#                     'Lyrics': lyrics
#                 })
#     else:
#         print("No artist found with the name:", artist_name)

# # Create DataFrame from track data
# track_df1 = pd.DataFrame(track_data)

# # Print the DataFrame
# print(track_df1)
# print(track_df1.shape)

Max Retries reached


Waiting for 10 seconds before retrying...


KeyboardInterrupt: 

In [108]:
# import spotipy
# from spotipy.oauth2 import SpotifyClientCredentials 
# import time

# client_id = "80230197a428480cbf1c4643273c48ab"
# client_secret = "38d1462bf0ec4f10871ae9c70e418d5b"

# # Initialize Spotify client
# client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
# sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# # Function to retrieve audio features with retries and exponential backoff
# def get_audio_features_with_retries(sp, track_id):
#     retries = 0
#     while retries < max_retries:
#         try:
#             # Attempt to retrieve audio features for the track
#             audio_features = sp.audio_features(track_id)[0]  # Assuming the first track in the list
#             return audio_features
#         except spotipy.SpotifyException as e:
#             # Check if the error is due to rate limiting (status code 429)
#             if e.http_status == 429:
#                 # Calculate backoff time
#                 backoff_time = 2 ** retries * sleep_time
#                 print(f"Waiting for {backoff_time} seconds before retrying...")
#                 time.sleep(backoff_time)
#                 retries += 1
#             else:
#                 # Re-raise the exception if it's not a rate limit error
#                 raise e
#     # Return None if maximum retries exceeded
#     return None

# # Maximum number of retries
# max_retries = 5

# # Sleep time between retries (in seconds)
# sleep_time = 10

# # Function to search for song lyrics with retries
# def search_song_with_retries(genius, song_name, artist_name):
#     retries = 0
#     while retries < max_retries:
#         try:
#             # Attempt to search for the song
#             song = genius.search_song(song_name, artist_name)
#             if song:
#                 return song.lyrics
#         except Exception as e:
#             print("Error occurred:", e)
#         # Song not found or error occurred, calculate backoff time
#         backoff_time = 2 ** retries * sleep_time
#         print(f"Waiting for {backoff_time} seconds before retrying...")
#         time.sleep(backoff_time)
#         retries += 1
#     return None

# # Populate the DataFrame with track information
# for track in top_tracks['tracks'][:10]:
#     # Get audio features for the track with retries
#     audio_features = get_audio_features_with_retries(sp, track['id'])
    
#     # If audio features are retrieved successfully, continue processing
#     if audio_features:
#         # Search for the lyrics with retries
#         lyrics = search_song_with_retries(genius, track['name'], artist_name)

#         # Append track information to list
#         track_data.append({
#             'Artist': artist_name,
#             'Track': track['name'],
#             'Danceability': audio_features['danceability'],
#             'Energy': audio_features['energy'],
#             'Key': audio_features['key'],
#             'Loudness': audio_features['loudness'],
#             'Mode': audio_features['mode'],
#             'Speechiness': audio_features['speechiness'],
#             'Acousticness': audio_features['acousticness'],
#             'Instrumentalness': audio_features['instrumentalness'],
#             'Liveness': audio_features['liveness'],
#             'Valence': audio_features['valence'],
#             'Tempo': audio_features['tempo'],
#             'Duration_ms': audio_features['duration_ms'],
#             'Lyrics': lyrics
#         })

# # Assuming you have defined and populated 'top_tracks' and 'genius' variables earlier


Searching for "Cruel Summer" by Taylor Swift...
Done.


Max Retries reached


Waiting for 10 seconds before retrying...


Max Retries reached


Waiting for 20 seconds before retrying...


Max Retries reached


Waiting for 40 seconds before retrying...


KeyboardInterrupt: 

In [ ]:
# artist_list = [
#     "Taylor Swift", "Drake", "Bad Bunny", "The Weeknd", "Ed Sheeran",
#     "Ariana Grande", "Justin Bieber", "Eminem", "Post Malone", "BTS",
#     "Kanye West", "J Balvin", "Billie Eilish", "Coldplay", "Travis Scott",
#     "Dua Lipa", "Juice WRLD", "Rihanna", "Imagine Dragons", "XXXTENTACION",
#     "Ozuna", "David Guetta", "Maroon 5", "Bruno Mars", "Khalid",
#     "Kendrick Lamar", "Shawn Mendes", "Calvin Harris", "Sam Smith",
#     "Lana Del Rey", "Beyoncé", "Daddy Yankee", "KAROL G", "Queen",
#     "Harry Styles", "Future", "Lady Gaga", "One Direction", "Nicki Minaj",
#     "J. Cole", "Adele", "Rauw Alejandro", "Anuel AA", "Chris Brown",
#     "Shakira", "The Beatles", "The Chainsmokers", "Selena Gomez",
#     "Linkin Park", "Arctic Monkeys", "Doja Cat", "Lil Uzi Vert", "Sia",
#     "SZA", "Marshmello", "Maluma", "Halsey", "Katy Perry", "Metro Boomin",
#     "21 Savage", "Miley Cyrus", "Olivia Rodrigo", "Twenty One Pilots",
#     "Avicii", "Arijit Singh", "Red Hot Chili Peppers", "Lil Baby", "Kygo",
#     "Feid", "Farruko", "OneRepublic", "Camila Cabello", "Myke Towers",
#     "Morgan Wallen", "Michael Jackson", "Peso Pluma", "Jason Derulo",
#     "Tyler, The Creator", "$uicideboy$", "Pitbull", "Mac Miller",
#     "Pritam", "Elton John", "Demi Lovato", "Metallica", "Pop Smoke",
#     "BLACKPINK", "Frank Ocean", "Tiësto", "Ellie Goulding", "DJ Snake",
#     "Alan Walker", "Lil Wayne", "The Neighbourhood", "James Arthur",
#     "DaBaby", "YoungBoy Never Broke Again", "AC/DC", "Charlie Puth",
#     "Lil Nas X"
# ]

In [ ]:
# for n in range(len(tracks['items'])): 
#         spotify_albums[album]['album'].append(artist_album_names[album_count]) 
#         spotify_albums[album]['track_number'].append(tracks['items'][n]['track_number'])
#         spotify_albums[album]['id'].append(tracks['items'][n]['id'])
#         spotify_albums[album]['name'].append(tracks['items'][n]['name'])
#         spotify_albums[album]['uri'].append(tracks['items'][n]['uri'])

In [40]:
# artist_name = []
# track_name = []
# track_popularity = []
# artist_id = []
# track_id = []
# for i in range(0,100):
#     track_results = sp.search(q='year:2022', type='track', limit=50,offset=i)
#     for i, t in enumerate(track_results['tracks']['items']):
#         artist_name.append(t['artists'][0]['name'])
#         artist_id.append(t['artists'][0]['id'])
#         track_name.append(t['name'])
#         track_id.append(t['id'])
#         track_popularity.append(t['popularity'])


In [46]:
# import pandas as pd
# track_df = pd.DataFrame({'artist_name' : artist_name, 'track_name' : track_name, 'track_id' : track_id, 'track_popularity' : track_popularity, 'artist_id' : artist_id})
# print(track_df.shape)
# track_df_pop = track_df.sort_values("track_popularity", ascending=False)

# track_df_pop
# # track_df_pop[track_df_pop['track_popularity'>str(0)]]

(5000, 5)


,artist_name,track_name,track_id,track_popularity,artist_id
0,Filippo Manni,Rarefatto,0k8GeGYuWugaEZTjwa9HCD,0,1Oh2EOPS64MOLd9qrBxRDI
3330,Alegre Musica de Piano,Easy (Music),22dVp4dfwfPwXhNqVPwgp7,0,16owCpT22GIGpzsw1otgJZ
3337,Rain Sounds XLE Library,Mindful Mysterious Monsoon,6eh2TGzATDaCryVol4PzdZ,0,0MzLFY17BkARAA3W94HeCI
3336,Timo Hensen,Hoofdstuk 6.5 - Ga Langs Start,2u0i01NDX4X2sWBc1jKBpT,0,39ACItGbfd4E1u7u1566JY
3335,Laura,AN Tagen wie heute (Akustikversion),3LxdewKB6V5JnVXlzvb5mD,0,6GZDqSPuebuAP0N85Lg2a8
...,...,...,...,...,...
1666,Duchamp-Killer,The love for the unknow,3i1PyeNXo6kl2G61jXcM72,0,0nm0A6y9ZuBENPbEKieDEu
1665,Relaxing Music by Vince Villin,"Relaxing Music, Pt. 41",10Jj4EOgyPLAZHDMC375lq,0,0XcHKSGQQ8BJhlFZ8mybEZ
1664,Jeffrey Neiman,Just While It Rains,6FBhAc7VPkMJqt9ok8srBk,0,04wnhasVv7aR3AS1ZPKaJE
1663,Alfrēds Kalniņš,Pie Staburaga,5SRg6JHeP1swl2kfR2L4E8,0,4LvkUxp6aEdTpOyS3Z3dmE


In [ ]:
# track_df_pop_no_d = track_df_pop.drop_duplicates("track_id")
# print(track_df_pop_no_d.shape)
# track_df_pop_no_d.head(10)

In [ ]:

# artist_popularity = []
# artist_genres = []
# artist_followers = []
# for a_id in track_df_pop_no_d.artist_id:
#   artist = sp.artist(a_id)
#   artist_popularity.append(artist['popularity'])
#   artist_genres.append(artist['genres'])
#   artist_followers.append(artist['followers']['total'])
  
  
# track_df_pop_no_d = track_df_pop_no_d.assign(artist_popularity=artist_popularity, artist_genres=artist_genres, artist_followers=artist_followers)
# track_df_pop_no_d.head(10)

In [33]:
# track_df_pop

,artist_name,track_name,track_id,track_popularity,artist_id
61,Casa Ambiente,La Mamola,6MBxBqobRWeDHkir4015lY,6,1QOITz2Xld7yktHlVeKW9m
21,iii,êverie Again- A New Start,0mNRcxPGUWrIozf1jis7PO,4,3pQeUO1dZJCsibkawRfUYT
94,Mahaveer Dabariya,Bhabhi Ke Pyar Me,5Hdc083tYk4b0i1W7eNJOm,2,2KoqXZyD05jakDuzP5bGTs
0,Jazz For Sleep,Alluring Music for Cocktails after Dinner,5KsDaCcYp2ebwwYfnxZE77,0,7ozAj8NWJM3EFOngEPryp3
64,Nursery Rhymes ABC,See-Saw,5whnS8L8WUMP5Hp0nDLyjV,0,6HwR10HBvWns7cEE7EJJTQ
...,...,...,...,...,...
31,Jeremiah Yamaoka,Rhabdomyoma Distensions,4Wu14o5KIlGNpryDzCMPxK,0,6yqn3czUxzfZPsIjeH7TCG
30,Brain Study Music Guys,Natural Radiance,1mOfUqZ3BP4AtvyUwrBVAL,0,3vyj4n7ADocniHDbKVMjZb
29,Entspannungsmusik Cornelia Verdin,Musik gegen Stress,7MUvw6fDbYvqOea7MFWtGR,0,1IbADom8509L3bCZN1kFAk
28,Cafe Music Deluxe,Friendly Ambiance for Concentration for Work,4IQwF2sA1mwNJw69wGIHa1,0,66rOnEXkD27GkyFRkK90nr


In [21]:
# import spotipy.util as util

# # Define the scope required (in this case, we want to access user's library)
# scope = 'user-library-read'
# redirect_uri='http://localhost:8888/callback'

# # Get the user's authorization
# username = '8ev9jb63b564s8oou11rgetbn'
# token = util.prompt_for_user_token(username, scope, client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri)

# artist_album_names = []
# artist_album_uris = []

# if token:
#     sp = spotipy.Spotify(auth=token)
#     # Retrieve the user's saved albums
#     results = sp.current_user_saved_albums()
#     # results = sp.user_playlist(user=username)
#     print(results)
#     for item in results['items']:
#         album = item['album']
#         # print(album['name'], 'by', album['artists'][0]['name'])
#         print(album['name'])
#         print("URL:", album['external_urls']['spotify'])  # Print the URL of the album
#         artist_album_names.append(album['name'])
#         artist_album_uris.append(album['external_urls']['spotify'])
# else:
#     print("Can't get token for", username)

{'href': 'https://api.spotify.com/v1/me/albums?offset=0&limit=20', 'items': [{'added_at': '2023-05-22T03:01:10Z', 'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3yKoHgjtyenCOIRaD2Gghu'}, 'href': 'https://api.spotify.com/v1/artists/3yKoHgjtyenCOIRaD2Gghu', 'id': '3yKoHgjtyenCOIRaD2Gghu', 'name': 'Cafe Music BGM channel', 'type': 'artist', 'uri': 'spotify:artist:3yKoHgjtyenCOIRaD2Gghu'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'BY', 'KZ', 'MD', 'UA', 'AL', 'BA', 'HR', 'ME', 'MK', 'RS',

In [22]:
# def album_songs(uri):
#     album = uri 
#     spotify_albums[album] = {}
#     #Create keys-values of empty lists inside nested dictionary for album
#     spotify_albums[album]['album'] = [] 
#     spotify_albums[album]['track_number'] = []
#     spotify_albums[album]['id'] = []
#     spotify_albums[album]['name'] = []
#     spotify_albums[album]['uri'] = []
#     #pull data on album tracks
#     tracks = sp.album_tracks(album) 
#     for n in range(len(tracks['items'])): 
#         spotify_albums[album]['album'].append(artist_album_names[album_count]) 
#         spotify_albums[album]['track_number'].append(tracks['items'][n]['track_number'])
#         spotify_albums[album]['id'].append(tracks['items'][n]['id'])
#         spotify_albums[album]['name'].append(tracks['items'][n]['name'])
#         spotify_albums[album]['uri'].append(tracks['items'][n]['uri'])

In [23]:
# spotify_albums = {}
# album_count = 0
# for i in artist_album_uris: #each album
#     album_songs(i)
#     print(str(artist_album_names[album_count]) + " album songs has been added to spotify_albums dictionary")
#     album_count+=1 #Updates album count once all tracks have been added

Coffee & Jazz album songs has been added to spotify_albums dictionary
An Evening With Silk Sonic album songs has been added to spotify_albums dictionary
THE BOOK album songs has been added to spotify_albums dictionary
THE BOOK 2 album songs has been added to spotify_albums dictionary
The Brightest Darkness album songs has been added to spotify_albums dictionary
Tokyo Cafe album songs has been added to spotify_albums dictionary
Live The Life album songs has been added to spotify_albums dictionary
Senses Inherited album songs has been added to spotify_albums dictionary


In [24]:
# def audio_features(album):
#     #Add new key-values to store audio features
#     spotify_albums[album]['acousticness'] = []
#     spotify_albums[album]['danceability'] = []
#     spotify_albums[album]['energy'] = []
#     spotify_albums[album]['instrumentalness'] = []
#     spotify_albums[album]['liveness'] = []
#     spotify_albums[album]['loudness'] = []
#     spotify_albums[album]['speechiness'] = []
#     spotify_albums[album]['tempo'] = []
#     spotify_albums[album]['valence'] = []
#     spotify_albums[album]['popularity'] = []
    
#     track_count = 0
#     for track in spotify_albums[album]['uri']:
#         #pull audio features per track
#         features = sp.audio_features(track)
        
#         #Append to relevant key-value
#         spotify_albums[album]['acousticness'].append(features[0]['acousticness'])
#         spotify_albums[album]['danceability'].append(features[0]['danceability'])
#         spotify_albums[album]['energy'].append(features[0]['energy'])
#         spotify_albums[album]['instrumentalness'].append(features[0]['instrumentalness'])
#         spotify_albums[album]['liveness'].append(features[0]['liveness'])
#         spotify_albums[album]['loudness'].append(features[0]['loudness'])
#         spotify_albums[album]['speechiness'].append(features[0]['speechiness'])
#         spotify_albums[album]['tempo'].append(features[0]['tempo'])
#         spotify_albums[album]['valence'].append(features[0]['valence'])
#         #popularity is stored elsewhere
#         pop = sp.track(track)
#         spotify_albums[album]['popularity'].append(pop['popularity'])
#         track_count+=1

In [11]:
# import time
# import numpy as np
# sleep_min = 2
# sleep_max = 5
# start_time = time.time()
# request_count = 0
# for i in spotify_albums:
#     audio_features(i)
#     request_count+=1
#     if request_count % 5 == 0:
#         print(str(request_count) + " playlists completed")
#         time.sleep(np.random.uniform(sleep_min, sleep_max))
#         print('Loop #: {}'.format(request_count))
#         print('Elapsed Time: {} seconds'.format(time.time() - start_time))

5 playlists completed
Loop #: 5
Elapsed Time: 35.95670294761658 seconds


In [12]:
# dic_df = {}
# dic_df['album'] = []
# dic_df['track_number'] = []
# dic_df['id'] = []
# dic_df['name'] = []
# dic_df['uri'] = []
# dic_df['acousticness'] = []
# dic_df['danceability'] = []
# dic_df['energy'] = []
# dic_df['instrumentalness'] = []
# dic_df['liveness'] = []
# dic_df['loudness'] = []
# dic_df['speechiness'] = []
# dic_df['tempo'] = []
# dic_df['valence'] = []
# dic_df['popularity'] = []
# for album in spotify_albums: 
#     for feature in spotify_albums[album]:
#         dic_df[feature].extend(spotify_albums[album][feature])
        
# len(dic_df['album'])

162

In [13]:
# import pandas as pd
# dataframe = pd.DataFrame.from_dict(dic_df)
# dataframe

,album,track_number,id,name,uri,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,Coffee & Jazz,1,33SawFo7Ld2TcTlRi6gmRu,Coffee & Jazz,spotify:track:33SawFo7Ld2TcTlRi6gmRu,0.937,0.659,0.2620,0.755000,0.1010,-18.118,0.0405,128.169,0.417,56
1,Coffee & Jazz,2,794hnOhksVsCLKiCq21eF4,Jazz Classic,spotify:track:794hnOhksVsCLKiCq21eF4,0.875,0.682,0.3700,0.651000,0.1110,-14.811,0.0296,133.994,0.484,53
2,Coffee & Jazz,3,3BVdEXx2x7r5G07VBl4KV1,Rock Time Jazz,spotify:track:3BVdEXx2x7r5G07VBl4KV1,0.883,0.750,0.2250,0.849000,0.1230,-18.360,0.0422,96.061,0.397,53
3,Coffee & Jazz,4,1dHbzdhZy1iIogqGFhCbDj,Peaceful,spotify:track:1dHbzdhZy1iIogqGFhCbDj,0.954,0.484,0.0289,0.950000,0.1250,-29.617,0.0394,128.272,0.204,53
4,Coffee & Jazz,5,1YsXY9MNq1XdNORrNYZiJb,Piano String,spotify:track:1YsXY9MNq1XdNORrNYZiJb,0.955,0.447,0.0670,0.910000,0.0972,-26.133,0.0423,121.935,0.126,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,Senses Inherited,9,6LakdrYLJyj5KXAhxHvBKr,傾慕,spotify:track:6LakdrYLJyj5KXAhxHvBKr,0.483,0.600,0.5500,0.000000,0.1170,-8.741,0.0318,136.026,0.504,25
158,Senses Inherited,10,3xNqmcJihjJX3EhN29HmOQ,歲月靜好,spotify:track:3xNqmcJihjJX3EhN29HmOQ,0.688,0.569,0.2750,0.000002,0.0974,-10.028,0.0416,131.922,0.172,33
159,Senses Inherited,11,5hdFOCoXHRbGEGF6A0IylT,裝睡的情人,spotify:track:5hdFOCoXHRbGEGF6A0IylT,0.877,0.544,0.4820,0.000009,0.1430,-8.939,0.0357,119.667,0.380,41
160,Senses Inherited,12,5ql4tlxS3GtADNsSRi2GnW,百年樹木,spotify:track:5ql4tlxS3GtADNsSRi2GnW,0.572,0.467,0.3490,0.000000,0.2040,-7.960,0.0339,140.212,0.343,46


In [ ]:
# 